In [18]:
import pandas as pd
import re
df = pd.read_csv('eColi_tRNA_modomics.csv')
df = pd.read_csv('sCere_tRNA_modomics.csv')

In [20]:
pd.set_option('display.max_colwidth', None)
newcolumns = ['tRNASequence' if n==5 else x.strip() for n, x in enumerate(df.columns)]
df.columns = newcolumns
for column in df.columns:
    print(column, end='---')
    

Type---AA type---Anticodon---Organism---Organellum---tRNASequence---MINTbase---

In [21]:
df[['AA type', 'Anticodon', 'tRNASequence']]

,AA type,Anticodon,tRNASequence
0,Ala,AGC,"-GGGCGUGU1-methylguanosine[show modification pathway]');return false"">KGCGUAGdihydrouridine[show modification pathway]');return false"">DC-GGdihydrouridine[show modification pathway]');return false"">D--AGCGCN2,N2-dimethylguanosine[show modification pathway]');return false"">RCUCCCUUinosine[show modification pathway]');return false"">IGC1-methylinosine[show modification pathway]');return false"">Opseudouridine[show modification pathway]');return false"">PGGGAGAG-------------------Gdihydrouridine[show modification pathway]');return false"">DCUCCGGTpseudouridine[show modification pathway]');return false"">PCGAUUCCGGACUCGUCCACCA"
1,Arg,UCU,"-GCUCGCGU1-methylguanosine[show modification pathway]');return false"">KN2-methylguanosine[show modification pathway]');return false"">LCGUAAdihydrouridine[show modification pathway]');return false"">D--GGC--AACGCN2,N2-dimethylguanosine[show modification pathway]');return false"">Rpseudouridine[show modification pathway]');return false"">PCUGACU5-methoxycarbonylmethyluridine[show modification pathway]');return false"">1CUN6-threonylcarbamoyladenosine[show modification pathway]');return false"">6Apseudouridine[show modification pathway]');return false"">PCAGAAG-------------------Adihydrouridine[show modification pathway]');return false"">DUAUGGGTpseudouridine[show modification pathway]');return false"">PCG1-methyladenosine[show modification pathway]');return false"">""CCCCCAUCGUGAGUGCCA"
2,Arg,ICG,"-pseudouridine[show modification pathway]');return false"">PUCCUCGU1-methylguanosine[show modification pathway]');return false"">KN2-methylguanosine[show modification pathway]');return false"">LCCCAAdihydrouridine[show modification pathway]');return false"">D--GGdihydrouridine[show modification pathway]');return false"">DC-ACGGCN2,N2-dimethylguanosine[show modification pathway]');return false"">Rpseudouridine[show modification pathway]');return false"">PCUGGCUinosine[show modification pathway]');return false"">ICGAACCAGAAG-------------------Adihydrouridine[show modification pathway]');return false"">DU5-methylcytidine[show modification pathway]');return false"">?CAGGTpseudouridine[show modification pathway]');return false"">PCA1-methyladenosine[show modification pathway]');return false"">""GUCCUGGCGGGGAAGCCA"
3,Asn,GUU,"-GACUCCAUGN2-methylguanosine[show modification pathway]');return false"">LCCAAGdihydrouridine[show modification pathway]');return false"">Ddihydrouridine[show modification pathway]');return false"">D-GGdihydrouridine[show modification pathway]');return false"">Ddihydrouridine[show modification pathway]');return false"">D-AAGGCN2,N2-dimethylguanosine[show modification pathway]');return false"">RUGCGACUGUUN6-threonylcarbamoyladenosine[show modification pathway]');return false"">6Apseudouridine[show modification pathway]');return false"">PCGCAAG-------------------Adihydrouridine[show modification pathway]');return false"">D5-methylcytidine[show modification pathway]');return false"">?GUGAGTpseudouridine[show modification pathway]');return false"">PCA1-methyladenosine[show modification pathway]');return false"">""CCCUCACUGGGGUCGCCA"
4,Asp,GUC,"-UCCGUGAUAGUUpseudouridine[show modification pathway]');return false"">PAAdihydrouridine[show modification pathway]');return false"">D--GGdihydrouridine[show modification pathway]');return false"">DC-AGAAUGGGCGCpseudouridine[show modification pathway]');return false"">PUGUC1-methylguanosine[show modification pathway]');return false"">KCGUGCCAG-------------------A-U5-methylcytidine[show modification pathway]');return false"">?GGGGTpseudouridine[show modification pathway]');return false"">PCAAUUCCCCGUCGCGGAGCCA"
5,Cys,GCA,"-GCUCGUAUGGCGCAGdihydrouridine[show modification pathway]');return false"">D--GGdihydrouridine[show modification pathway]');return false"">D--AGCGCAGCAGApseudouridine[show modification pathway]');return false"">PUGCAN6-isopentenyladenosine[show modification pathway]');return false"">+Apseu

# Imma filter based on whether there is a m1A in SaccCer

In [29]:
filt = ( df['tRNASequence'].str.contains('1-methyladenosine') )

In [32]:
df[ ['AA type', 'Anticodon'] ].loc[filt]

,AA type,Anticodon
1,Arg,UCU
2,Arg,ICG
3,Asn,GUU
5,Cys,GCA
11,Ile,AAU
12,Ile,UAU
13,Leu,UAG
15,Leu,UAA
16,Lys,CUU
17,Lys,3UU


# And Imma filter based on whether there isn't an m1A in SaccCer

In [35]:
filt = [not x for x in filt]

In [36]:
df[ ['AA type', 'Anticodon'] ].loc[filt]

,AA type,Anticodon
0,Ala,AGC
4,Asp,GUC
6,Glu,UUC
7,Gly,GCC
8,Gly,UCC
9,His,GUG
10,His,GUG
14,Leu,CAA
22,Ser,CGA
23,Ser,UGA


In [82]:
tRNA = df.loc[0][5]
tRNA

'-GGGGCUAUAGCUCAGCdihydrouridine[show modification pathway]\');return false">D-GGG--AGAGCGCCUGCUUuridine 5-oxyacetic acid[show modification pathway]\');return false">VGCACGCAGGAG-------------------7-methylguanosine[show modification pathway]\');return false">7UCUGCGGTpseudouridine[show modification pathway]\');return false">PCGAUCCCGCAUAGCUCCACCA'

# Making a pattern to take out text
The plan is to make a regular expression that takes out that wierd text that shows up in front of modifications

In [56]:
#matches a lowercase letter or a number followed by -
#   p in pseudouridine or 7- in 7-methylguanosine
#need the #- because otherwise it would match the character
#MODOMICS uses to denote modifications in the code.

pattern = r'([a-z]|\d-).+?>'
pattern = re.compile(pattern)

In [112]:
def findAnnotation(tRNAText):
    '''
    This method uses the pattern above to search an 
    input string for modomics style annotation.
    '''
    matches = pattern.finditer(tRNAText)
    for match in matches:
        print (match)

# Headers freaking have spaces
The freaking MODOMICS headers have weird spaces so I needed to strip them.

# Difficult to pull info from df
Also for whatever reason I need to jump through hoops in order to iterate over the rows of the df and pull out the info I need. That is why the following code is so weird. All its doing is getting seq, aa, aCodon info and feeding sequence string to the pattern matching method.

In [114]:
for index in range(0, df[['tRNASequence', 'AA type', 'Anticodon']].shape[0]):
    seq, aa, aCodon = list( df[['tRNASequence', 'AA type', 'Anticodon']].loc[index] )
    print(aa, aCodon, '\n')
    findAnnotation(seq)
    print('\n\n')

Ala  VGC  

<re.Match object; span=(17, 75), match='dihydrouridine[show modification pathway]\');retu>
<re.Match object; span=(95, 163), match='uridine 5-oxyacetic acid[show modification pathwa>
<re.Match object; span=(194, 255), match='7-methylguanosine[show modification pathway]\');r>
<re.Match object; span=(264, 321), match='pseudouridine[show modification pathway]\');retur>



Ala  GGC  

<re.Match object; span=(17, 75), match='dihydrouridine[show modification pathway]\');retu>
<re.Match object; span=(126, 187), match='7-methylguanosine[show modification pathway]\');r>
<re.Match object; span=(196, 253), match='pseudouridine[show modification pathway]\');retur>



Arg  ICG  

<re.Match object; span=(8, 65), match='4-thiouridine[show modification pathway]\');retur>
<re.Match object; span=(74, 132), match='dihydrouridine[show modification pathway]\');retu>
<re.Match object; span=(137, 195), match='dihydrouridine[show modification pathway]\');retu>
<re.Match object; span=(208, 266), ma

# Recap
The pattern matching code seems to be working fine, now I need to find a way to have it excize the matches it finds. Maybe write a method that pulls the boundries and stitches them together into a final string.

## Example
'Hello world it is a beautiful day'
(6, 12) -- world 
(20, 30) -- beautiful

### excision
(0, 6)(12, 20)(30, 32)
'Hello it is a day'
